# Data Prep Notebook

This notebook was written to download Zillow data and merge is with U.S. census county geodata as well as area deprivation data.  The notebook was written to test data engineering code which will later be transferred to python scripts.

In [3]:
import pandas as pd
from google.cloud import bigquery
import pandas_gbq

from data_download_utilities import csv_download, bq_download

In [4]:
avi_df = bq_download('SELECT * FROM `bigquery-public-data.broadstreet_adi.area_deprivation_index_by_county` WHERE YEAR = 2020 AND state_fips_code = "48"')
avi_df

,geo_id,state_fips_code,county_fips_code,county_name,state_name,state,year,area_deprivation_index_percent
0,05000US48001,48,48001,Anderson County,Texas,TX,2020,70.0
1,05000US48003,48,48003,Andrews County,Texas,TX,2020,54.0
2,05000US48005,48,48005,Angelina County,Texas,TX,2020,80.0
3,05000US48007,48,48007,Aransas County,Texas,TX,2020,63.0
4,05000US48009,48,48009,Archer County,Texas,TX,2020,17.0
...,...,...,...,...,...,...,...,...
249,05000US48499,48,48499,Wood County,Texas,TX,2020,52.0
250,05000US48501,48,48501,Yoakum County,Texas,TX,2020,68.0
251,05000US48503,48,48503,Young County,Texas,TX,2020,60.0
252,05000US48505,48,48505,Zapata County,Texas,TX,2020,100.0


In [5]:
tx_geo_df = bq_download('SELECT * FROM `bigquery-public-data.geo_us_boundaries.counties` where state_fips_code = "48"')
tx_geo_df

,geo_id,state_fips_code,county_fips_code,county_gnis_code,county_name,lsad_name,lsad_code,fips_class_code,mtfcc_feature_class_code,csa_fips_code,cbsa_fips_code,met_div_fips_code,functional_status,area_land_meters,area_water_meters,int_point_lat,int_point_lon,int_point_geom,county_geom
0,48383,48,48383,01383977,Reagan,Reagan County,06,H1,G4020,None,None,None,A,3044049120,1792716,31.375191,-101.514395,POINT(-101.5143954 31.3751911),"POLYGON((-101.776176 31.081392, -101.776191 31..."
1,48461,48,48461,01384016,Upton,Upton County,06,H1,G4020,None,None,None,A,3215010598,464309,31.353816,-102.041546,POINT(-102.0415461 31.3538163),"POLYGON((-102.009444 31.082669, -102.000409 31..."
2,48089,48,48089,01383830,Colorado,Colorado County,06,H1,G4020,None,None,None,A,2487129887,34703995,29.596296,-96.508935,POINT(-96.5089347 29.596296),"POLYGON((-96.706155 29.486696, -96.705809 29.4..."
3,48337,48,48337,01383954,Montague,Montague County,06,H1,G4020,None,None,None,A,2411034607,19087361,33.678356,-97.725014,POINT(-97.7250138 33.6783558),"POLYGON((-97.978804 33.727526, -97.978805 33.7..."
4,48299,48,48299,01383935,Llano,Llano County,06,H1,G4020,None,None,None,A,2419185654,82370906,30.707584,-98.684690,POINT(-98.6846901 30.7075845),"POLYGON((-98.964016 30.705236, -98.964017 30.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,48145,48,48145,01383858,Falls,Falls County,06,H1,G4020,None,47380,None,A,1982545002,21693803,31.251930,-96.934128,POINT(-96.9341275 31.2519298),"POLYGON((-97.017623 31.410927, -97.018061 31.4..."
250,48485,48,48485,01384028,Wichita,Wichita County,06,H1,G4020,None,48660,None,A,1625459969,13644166,33.988213,-98.708010,POINT(-98.7080098 33.9882125),"POLYGON((-98.423546 34.018645, -98.423618 34.0..."
251,48077,48,48077,01383824,Clay,Clay County,06,H1,G4020,None,48660,None,A,2819870186,72508536,33.785904,-98.212917,POINT(-98.2129174 33.7859042),"POLYGON((-98.42308 33.944531, -98.423081 33.94..."
252,48009,48,48009,01383790,Archer,Archer County,06,H1,G4020,None,48660,None,A,2339508173,57767712,33.616305,-98.687267,POINT(-98.6872669 33.6163054),"POLYGON((-98.751999 33.395871, -98.750339 33.3..."


In [87]:
zillow_df = csv_download('https://files.zillowstatic.com/research/public_csvs/zhvi/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1719616688')

# Subset on Texas
zillow_tx_df = zillow_df[zillow_df['State'] == 'TX']
zillow_tx_df['StateCodeFIPS'] = zillow_tx_df['StateCodeFIPS'].astype(str)
zillow_tx_df['MunicipalCodeFIPS'] = zillow_tx_df['MunicipalCodeFIPS'].astype(str)
zillow_tx_df['StateCodeFIPS'] = zillow_tx_df['StateCodeFIPS'].apply('{:0>2}'.format)
zillow_tx_df['MunicipalCodeFIPS'] = zillow_tx_df['MunicipalCodeFIPS'].apply('{:0>3}'.format)
zillow_tx_df['county_fips_code'] =  zillow_tx_df['StateCodeFIPS'] + zillow_tx_df['MunicipalCodeFIPS']

months_to_avg_list = []
for n in zillow_tx_df.columns:
    if n[0:4] == '2020':
        print(n)
        months_to_avg_list.append(n)
zillow_tx_df['average'] = zillow_tx_df[months_to_avg_list].mean(axis=1)
# Need to drop State column to avoid a conflict in variable names when merging:
zillow_tx_df.drop('State', axis = 1, inplace = True)

zillow_tx_df

2020-01-31
2020-02-29
2020-03-31
2020-04-30
2020-05-31
2020-06-30
2020-07-31
2020-08-31
2020-09-30
2020-10-31
2020-11-30
2020-12-31


/var/tmp/ipykernel_5741/4079113788.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zillow_tx_df['StateCodeFIPS'] = zillow_tx_df['StateCodeFIPS'].astype(str)
/var/tmp/ipykernel_5741/4079113788.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zillow_tx_df['MunicipalCodeFIPS'] = zillow_tx_df['MunicipalCodeFIPS'].astype(str)
/var/tmp/ipykernel_5741/4079113788.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,RegionID,SizeRank,RegionName,RegionType,StateName,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,2000-02-29,...,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,county_fips_code,average
2,1090,2,Harris County,county,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,111014.888728,110986.387143,...,284948.059503,284693.543837,284398.217589,284510.879505,284979.598951,285995.201590,287052.205088,287651.772843,48201,211154.501333
8,978,8,Dallas County,county,TX,"Dallas-Fort Worth-Arlington, TX",48,113,96618.812191,96672.070463,...,319988.065224,320046.888242,319942.293815,320142.012121,320972.695634,322460.881161,323551.067248,323804.466510,48113,226971.701749
14,2049,14,Tarrant County,county,TX,"Dallas-Fort Worth-Arlington, TX",48,439,114637.326313,114709.951201,...,332494.890444,332383.098229,332214.432397,332252.111874,332354.308134,332785.323754,333207.064646,333278.929812,48439,236090.856583
15,386,15,Bexar County,county,TX,"San Antonio-New Braunfels, TX",48,029,103545.511714,103643.697618,...,272190.337605,270916.701106,269414.366137,268324.083911,267761.822952,267935.129626,268271.775846,268246.850043,48029,203772.741571
33,1440,33,Travis County,county,TX,"Austin-Round Rock-Georgetown, TX",48,453,189092.292447,189710.624810,...,535337.368601,531653.587084,527761.861210,524869.794009,522741.348936,521926.791189,521594.286470,520677.306212,48453,389539.163102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,1247,3171,Motley County,county,TX,NaN,48,345,NaN,NaN,...,97666.006365,95097.426710,92690.144412,91123.621370,91385.345818,93333.151040,95090.838602,96401.772114,48345,NaN
3045,1557,3172,Briscoe County,county,TX,NaN,48,045,NaN,NaN,...,94201.503184,91538.637597,88716.298862,86654.005172,85607.267205,87035.371281,89635.100699,91648.040876,48045,NaN
3048,2706,3175,Glasscock County,county,TX,NaN,48,173,NaN,NaN,...,245918.050851,245825.731384,243853.915180,241233.493107,239326.761923,239364.009739,239743.736982,238623.236228,48173,NaN
3050,518,3178,Foard County,county,TX,NaN,48,155,NaN,NaN,...,122837.341055,120325.111516,118079.178118,115680.460125,115489.163983,117052.599035,120253.590191,123255.984980,48155,NaN


In [88]:
for idx, row in zillow_tx_df.iterrows():
    print(row['average'])

211154.5013328416
226971.701748903
236090.8565833268
203772.74157069877
389539.16310245596
345461.66017883143
322238.81122762314
122821.77341520292
146273.18359134902
274452.55530934146
259260.05924614155
310056.6435223363
126514.00277768057
251194.86322179824
178427.15170930882
187261.20517396546
243612.98789133978
160499.49031583485
173501.58896799487
180704.6824735836
141312.25141739537
293644.3365266605
189335.26601093254
234972.26146312323
261394.573958442
237154.39625906924
246783.03051848998
284354.0411425837
221090.80230085095
331838.3626804373
314897.42165889725
150603.48563572412
235542.42758732478
183232.3663397604
197901.82120249874
111817.91450500785
172365.67744641614
179235.94446508633
99792.87745264295
298669.62933505204
206925.41208225884
260428.13409898468
144097.89916446203
166099.61651446263
191370.85187200867
151449.39552506083
173048.5466704354
198799.58310364847
153543.99886890672
210642.42555318706
170648.0598459872
186788.7937924289
277651.1612451964
105264.982

In [89]:
zillow_tx_df.dtypes

RegionID              int64
SizeRank              int64
RegionName           object
RegionType           object
StateName            object
                     ...   
2024-03-31          float64
2024-04-30          float64
2024-05-31          float64
county_fips_code     object
average             float64
Length: 303, dtype: object

In [90]:
tx_geo_df.dtypes

geo_id                       object
state_fips_code              object
county_fips_code             object
county_gnis_code             object
county_name                  object
lsad_name                    object
lsad_code                    object
fips_class_code              object
mtfcc_feature_class_code     object
csa_fips_code                object
cbsa_fips_code               object
met_div_fips_code            object
functional_status            object
area_land_meters              Int64
area_water_meters             Int64
int_point_lat               float64
int_point_lon               float64
int_point_geom               object
county_geom                  object
dtype: object

In [91]:
avi_df.dtypes

geo_id                             object
state_fips_code                    object
county_fips_code                   object
county_name                        object
state_name                         object
state                              object
year                                Int64
area_deprivation_index_percent    float64
dtype: object

In [92]:
merged_df = pd.merge(tx_geo_df, avi_df, on='county_fips_code', how = 'outer')

merged_df

,geo_id_x,state_fips_code_x,county_fips_code,county_gnis_code,county_name_x,lsad_name,lsad_code,fips_class_code,mtfcc_feature_class_code,csa_fips_code,...,int_point_lon,int_point_geom,county_geom,geo_id_y,state_fips_code_y,county_name_y,state_name,state,year,area_deprivation_index_percent
0,48383,48,48383,01383977,Reagan,Reagan County,06,H1,G4020,None,...,-101.514395,POINT(-101.5143954 31.3751911),"POLYGON((-101.776176 31.081392, -101.776191 31...",05000US48383,48,Reagan County,Texas,TX,2020,57.0
1,48461,48,48461,01384016,Upton,Upton County,06,H1,G4020,None,...,-102.041546,POINT(-102.0415461 31.3538163),"POLYGON((-102.009444 31.082669, -102.000409 31...",05000US48461,48,Upton County,Texas,TX,2020,79.0
2,48089,48,48089,01383830,Colorado,Colorado County,06,H1,G4020,None,...,-96.508935,POINT(-96.5089347 29.596296),"POLYGON((-96.706155 29.486696, -96.705809 29.4...",05000US48089,48,Colorado County,Texas,TX,2020,59.0
3,48337,48,48337,01383954,Montague,Montague County,06,H1,G4020,None,...,-97.725014,POINT(-97.7250138 33.6783558),"POLYGON((-97.978804 33.727526, -97.978805 33.7...",05000US48337,48,Montague County,Texas,TX,2020,60.0
4,48299,48,48299,01383935,Llano,Llano County,06,H1,G4020,None,...,-98.684690,POINT(-98.6846901 30.7075845),"POLYGON((-98.964016 30.705236, -98.964017 30.7...",05000US48299,48,Llano County,Texas,TX,2020,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,48145,48,48145,01383858,Falls,Falls County,06,H1,G4020,None,...,-96.934128,POINT(-96.9341275 31.2519298),"POLYGON((-97.017623 31.410927, -97.018061 31.4...",05000US48145,48,Falls County,Texas,TX,2020,95.0
250,48485,48,48485,01384028,Wichita,Wichita County,06,H1,G4020,None,...,-98.708010,POINT(-98.7080098 33.9882125),"POLYGON((-98.423546 34.018645, -98.423618 34.0...",05000US48485,48,Wichita County,Texas,TX,2020,67.0
251,48077,48,48077,01383824,Clay,Clay County,06,H1,G4020,None,...,-98.212917,POINT(-98.2129174 33.7859042),"POLYGON((-98.42308 33.944531, -98.423081 33.94...",05000US48077,48,Clay County,Texas,TX,2020,33.0
252,48009,48,48009,01383790,Archer,Archer County,06,H1,G4020,None,...,-98.687267,POINT(-98.6872669 33.6163054),"POLYGON((-98.751999 33.395871, -98.750339 33.3...",05000US48009,48,Archer County,Texas,TX,2020,17.0


In [93]:
merged_df = pd.merge(merged_df, zillow_tx_df, on='county_fips_code', how = 'outer')

merged_df

,geo_id_x,state_fips_code_x,county_fips_code,county_gnis_code,county_name_x,lsad_name,lsad_code,fips_class_code,mtfcc_feature_class_code,csa_fips_code,...,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,average
0,48383,48,48383,01383977,Reagan,Reagan County,06,H1,G4020,None,...,151593.594208,153884.130270,154432.645370,153434.422649,151729.337150,151166.159237,152079.264063,155091.693308,157837.008770,NaN
1,48461,48,48461,01384016,Upton,Upton County,06,H1,G4020,None,...,97675.480157,97826.160360,97581.650623,96527.286018,94936.375187,93488.987551,93415.716682,93958.716830,95498.212471,114668.017466
2,48089,48,48089,01383830,Colorado,Colorado County,06,H1,G4020,None,...,237368.132749,237187.707142,236523.842733,235143.246022,234457.291839,234680.596399,237358.318891,240412.860653,242258.985961,194684.853214
3,48337,48,48337,01383954,Montague,Montague County,06,H1,G4020,None,...,222990.742347,221881.491042,221692.064806,220675.994076,219564.056322,218880.150997,219893.886908,221887.379342,222742.261632,162273.463390
4,48299,48,48299,01383935,Llano,Llano County,06,H1,G4020,None,...,506123.719839,501978.769786,497002.974799,491334.084753,487284.663999,484163.205366,483355.589722,483154.602264,483481.366353,346568.566084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,48145,48,48145,01383858,Falls,Falls County,06,H1,G4020,None,...,128857.470890,127088.633373,125196.059188,123473.109204,123384.143036,123696.463221,124739.776870,126073.132672,126995.546846,116745.560985
250,48485,48,48485,01384028,Wichita,Wichita County,06,H1,G4020,None,...,159456.957189,159870.686556,160214.134584,160189.635075,160173.271505,160560.097609,161370.821090,162135.575237,162693.609277,111817.914505
251,48077,48,48077,01383824,Clay,Clay County,06,H1,G4020,None,...,180736.334171,179660.508860,178532.175357,177373.737730,177092.953193,177849.439200,178827.929903,180087.661424,180450.047225,142894.023659
252,48009,48,48009,01383790,Archer,Archer County,06,H1,G4020,None,...,212070.069320,211676.261763,210522.277219,209468.769280,209349.763630,210636.005873,213091.853950,215936.912464,217877.665857,166031.803063


In [94]:
for n in merged_df.columns:
    if n[0].isdigit():
        print(f'Deleting {n} colummn.')
        merged_df.drop(n, axis = 1, inplace = True)
merged_df

Deleting 2000-01-31 colummn.
Deleting 2000-02-29 colummn.
Deleting 2000-03-31 colummn.
Deleting 2000-04-30 colummn.
Deleting 2000-05-31 colummn.
Deleting 2000-06-30 colummn.
Deleting 2000-07-31 colummn.
Deleting 2000-08-31 colummn.
Deleting 2000-09-30 colummn.
Deleting 2000-10-31 colummn.
Deleting 2000-11-30 colummn.
Deleting 2000-12-31 colummn.
Deleting 2001-01-31 colummn.
Deleting 2001-02-28 colummn.
Deleting 2001-03-31 colummn.
Deleting 2001-04-30 colummn.
Deleting 2001-05-31 colummn.
Deleting 2001-06-30 colummn.
Deleting 2001-07-31 colummn.
Deleting 2001-08-31 colummn.
Deleting 2001-09-30 colummn.
Deleting 2001-10-31 colummn.
Deleting 2001-11-30 colummn.
Deleting 2001-12-31 colummn.
Deleting 2002-01-31 colummn.
Deleting 2002-02-28 colummn.
Deleting 2002-03-31 colummn.
Deleting 2002-04-30 colummn.
Deleting 2002-05-31 colummn.
Deleting 2002-06-30 colummn.
Deleting 2002-07-31 colummn.
Deleting 2002-08-31 colummn.
Deleting 2002-09-30 colummn.
Deleting 2002-10-31 colummn.
Deleting 2002-

,geo_id_x,state_fips_code_x,county_fips_code,county_gnis_code,county_name_x,lsad_name,lsad_code,fips_class_code,mtfcc_feature_class_code,csa_fips_code,...,area_deprivation_index_percent,RegionID,SizeRank,RegionName,RegionType,StateName,Metro,StateCodeFIPS,MunicipalCodeFIPS,average
0,48383,48,48383,01383977,Reagan,Reagan County,06,H1,G4020,None,...,57.0,1353.0,3015.0,Reagan County,county,TX,NaN,48,383,NaN
1,48461,48,48461,01384016,Upton,Upton County,06,H1,G4020,None,...,79.0,780.0,3018.0,Upton County,county,TX,NaN,48,461,114668.017466
2,48089,48,48089,01383830,Colorado,Colorado County,06,H1,G4020,None,...,59.0,2209.0,1846.0,Colorado County,county,TX,NaN,48,089,194684.853214
3,48337,48,48337,01383954,Montague,Montague County,06,H1,G4020,None,...,60.0,2442.0,1882.0,Montague County,county,TX,NaN,48,337,162273.463390
4,48299,48,48299,01383935,Llano,Llano County,06,H1,G4020,None,...,41.0,603.0,1812.0,Llano County,county,TX,NaN,48,299,346568.566084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,48145,48,48145,01383858,Falls,Falls County,06,H1,G4020,None,...,95.0,509.0,2029.0,Falls County,county,TX,"Waco, TX",48,145,116745.560985
250,48485,48,48485,01384028,Wichita,Wichita County,06,H1,G4020,None,...,67.0,2095.0,500.0,Wichita County,county,TX,"Wichita Falls, TX",48,485,111817.914505
251,48077,48,48077,01383824,Clay,Clay County,06,H1,G4020,None,...,33.0,132.0,2469.0,Clay County,county,TX,"Wichita Falls, TX",48,077,142894.023659
252,48009,48,48009,01383790,Archer,Archer County,06,H1,G4020,None,...,17.0,835.0,2603.0,Archer County,county,TX,"Wichita Falls, TX",48,009,166031.803063


In [95]:
# Construct a BigQuery client object.
client = bigquery.Client()

project_id = "even-gearbox-427900-t9"
dataset_name = "avi_housing"

# TODO(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = f'{project_id}.{dataset_name}'.format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/even-gearbox-427900-t9/datasets?prettyPrint=false: Already Exists: Dataset even-gearbox-427900-t9:avi_housing

In [101]:
# TODO: Set project_id to your Google Cloud Platform project ID.
# project_id = "my-project"
# TODO: Set table_id to the full destination table ID (including the
#       dataset ID).
table_id = 'merged_data'
table_id_combined = f'{dataset_name}.{table_id}'


pandas_gbq.to_gbq(merged_df, table_id_combined, project_id=project_id, table_schema=[{'name': 'county_geom', 'type': 'GEOGRAPHY'}])

100%|██████████| 1/1 [00:00<00:00, 8924.05it/s]


In [81]:
for n in merged_df.columns.to_lower().sort_values():
    print(n)

AttributeError: 'Index' object has no attribute 'to_lower'

In [99]:
data = merged_df
data.columns = [x.lower() for x in data.columns]
for n in data.columns.sort_values():
    print(n)

area_deprivation_index_percent
area_land_meters
area_water_meters
average
cbsa_fips_code
county_fips_code
county_geom
county_gnis_code
county_name_x
county_name_y
csa_fips_code
fips_class_code
functional_status
geo_id_x
geo_id_y
int_point_geom
int_point_lat
int_point_lon
lsad_code
lsad_name
met_div_fips_code
metro
mtfcc_feature_class_code
municipalcodefips
regionid
regionname
regiontype
sizerank
state
state_fips_code_x
state_fips_code_y
state_name
statecodefips
statename
year


In [102]:
for n in avi_df.columns:
    print(n)
    

geo_id
state_fips_code
county_fips_code
county_name
state_name
state
year
area_deprivation_index_percent
